## Session Overview

In this session, we will:

- Run Ollama locally and explore its capabilities.
- Install the `llama3:2` (latest) model.
- Compare the outputs of three language models: **GPT**, **Gemini**, and **Llama**.
- Use GPT as a judge to evaluate the outputs (LLM-as-judge approach).
- Explore generating structured outputs using **Pydantic**.

## Prerequisites

- **Install Ollama**: Follow the instructions at [Ollama's official website](https://ollama.com/download) to install Ollama on your machine.
- **Download the Llama 3 model**: Run the following command in your terminal to download and start the Llama 3 model:
  
  ```
  ollama run llama3.2:1b
  ```

  This will ensure the model is available locally for comparison.
- **Gemini API Key (Optional)**: Sign-up on Gemini for a free account, create an API key and store it in your .env file as `GOOGLE_API_KEY`. See `.env.example` for reference.

In [ ]:
# import necessary libraries
from openai import OpenAI
from dotenv import load_dotenv

In [ ]:
# Load env variables and initiate clients for OpenAI, Gemini and Ollama (llama)

# Gemini Base URL: https://generativelanguage.googleapis.com/v1beta/openai/
# Gemini Model: gemini-2.0-flash
# Gemini Models: https://ai.google.dev/gemini-api/docs/models

# Ollama Base URL: http://localhost:11434/v1
# Ollama Model: llama3.2

load_dotenv(override=True)
openai_client = OpenAI()

In [ ]:
# Ask GPT to generate a nuanced question that can be asked to GPT, Gemini and Ollama to judge their capabilities
# Prompt: Please come up with a challenging, nuanced question that can be asked to an LLM to evaluate its intelligence. Answer only with a question, no explanation

In [ ]:
# Prepare messages for LLMs with question to answer

In [ ]:
# Ask GPT to answer the question

In [ ]:
# Ask Gemini to answer the question

In [ ]:
# Ask Ollama to answer the question

In [ ]:
# Write a prompt to ask GPT 4.1 to judge the answers based on clarity and strength of the argument.
# GPT should respond with a json object.

In [ ]:
# Send judge prompt to GPT 4.1

In [ ]:
# Parse judge output to json

### Is there a better way to obtain structured output?

Yes, you can use **Pydantic** to define a schema for the expected output and validate the response, ensuring it adheres to the desired structure.

In [ ]:
# define pydantic schema for judge's structured output

In [ ]:
# Rewrite the judge prompt. This time, without the json object.

In [ ]:
# Send judge prompt to GPT 4.1 again
# Instead of Chat Completions create, use parse this time